## Paso 1. Importo librerias

In [1]:
import os
from IPython.display import Markdown

# Librerías para la preparación de datos
from langchain.document_loaders import PyPDFDirectoryLoader  # libreria para cargar pdf
from langchain.text_splitter import RecursiveCharacterTextSplitter  # libreria para hacer chunks
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings

# Librerías para el proceso de Retrieval
from langchain import hub  # si no tenemos esto, no podemos importar el langchain, 
from langchain_core.output_parsers import StrOutputParser # para que la clase de gemini, me devuelva el texto parseado
from langchain_core.runnables import RunnablePassthrough # funcion aux que tiene que tener la entrada de la cadena con contexto de la pregunta siempre = True
from langchain_google_genai import ChatGoogleGenerativeAI # el llm q estamos usando 

## Paso 2. Configuración inicial de carpetas

Crearemos dos carpetas:

- MisDatos: Aquí se almacenarán los archivos adicionales que utilizaremos para ampliar la base de conocimiento del modelo.
- VectorDB: En esta carpeta se almacenará la base de datos Vectorial.

In [2]:
import os
os.mkdir("content")
os.mkdir("content/MisDatos")
os.mkdir("content/VectorDB")

Carga en la carpeta MisDatos, los PDFs que quieres utilizar para personalizar las respuestas generadas.

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
os.environ["LANGCHAIN_TRACING_V2"] = "true" 

## Paso 3. Preparación de los datos

Leeremos los archivos PDF de la carpeta Mis Datos y los convertiremos en embeddings.

In [12]:
source_data_folder = "./content/MisDatos"
# Leyendo los PDFs del directorio configurado --> cargamos todo lo q haya, genera una lista con 1 pag x PDF, las img las esta obviando
loader = PyPDFDirectoryLoader(source_data_folder)
data_on_pdf = loader.load()
# cantidad de data cargada
len(data_on_pdf)

# Particionando los datos. Con un tamaño delimitado (chunks) y 
# 200 caracters de overlapping para preservar el contexto
text_splitter = RecursiveCharacterTextSplitter(    # no pilla strings, es un objeto de tipo loader.load
    separators=["\n\n", "\n", ". ", " "],
    chunk_size=1000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(data_on_pdf)
# Cantidad de chunks obtenidos
len(splits)

108

In [13]:
data_on_pdf

[Document(metadata={'source': 'content\\MisDatos\\MDRH2011M06U29(2).pdf', 'page': 0}, page_content='1\n 1. La dirección de recursos humanos como facilitadora del cambio  ........2\n  1.1.  Roles y funciones para RRHH  .....................................................................................2\n  1.2. Niveles de cambio  ...............................................................................................................5\n\t 2.\t\tFactores\tque\tinfluyen\ten\tprocesos\tde\tcambio\torganizacional \t ..........8\n  2.1.  Estilos de dirección, liderazgo, talento y modelos \n   de desarrollo para el cambio  ......................................................................................8\n  2.2. Modelos organizativos  ...................................................................................................11\n  2.3. Comunicación interna y cambio  .............................................................................13\n\t 3.\t\tCaracterísticas\tde

In [8]:
# pip install cohere

Note: you may need to restart the kernel to use updated packages.


In [14]:
from langchain.vectorstores import Chroma
from langchain.embeddings import CohereEmbeddings

# Crea la instancia de embeddings con Cohere
embeddings_model = CohereEmbeddings(cohere_api_key=os.environ["COHERE_API_KEY"], user_agent="antonio")  #user_agent por alguna razon lo pide, le ponemos cualquiera para que no de error 

path_db = "./content/VectorDB"  # Ruta a la base de datos del vector store

# Crear el vector store a partir de tus documentos 'splits'
vectorstore = Chroma.from_documents(   
    documents=splits, 
    embedding=embeddings_model, 
    persist_directory=path_db # persist = guardarlos: Para que podamos persistir los datos y no generarlos todo el tiempo
)

In [15]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=os.environ["GOOGLE_API_KEY"])

In [16]:
retriever = vectorstore.as_retriever(k=10)  # Por default son 4, 

In [17]:
prompt = hub.pull("rlm/rag-prompt")

In [18]:
def format_docs(docs):
    # Funcion auxiliar para enviar el contexto al modelo como parte del prompt
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [24]:
prompt.messages[0].prompt.template

"You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"

In [25]:
prompt.messages[0].prompt.template = "You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. Answer in spanish. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"

In [26]:
pregunta = "Que funcion actual tiene RRHH" 
response = rag_chain.invoke(pregunta)
Markdown(response)

La función actual de RRHH está evolucionando hacia un rol más estratégico y de consultoría interna.  Se centra en impulsar el talento, alinear los objetivos de la empresa con los de sus empleados y facilitar cambios organizativos que mejoren la competitividad.  Además, RRHH busca promover una cultura de innovación y desarrollo dentro de la empresa.
